In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[5]").appName("Processing").getOrCreate()

24/11/14 10:41:10 WARN Utils: Your hostname, Abhays-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.36 instead (on interface en0)
24/11/14 10:41:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/14 10:41:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv(path="bikeshare data/daily_rent_detail.csv", header=True, inferSchema=True)

In [4]:
futuredata = data.filter(col("started_at") > "2024-01-01") \
    .orderBy(col("started_at").asc())

In [16]:
weather = spark.read.csv(path="bikeshare data/weather.csv", header=True, inferSchema=True)

In [17]:
weather = weather.select("datetime", "tempmax", "tempmin", "humidity", "precip", "snow", "windspeed", "windgust", "conditions")

In [18]:
futureweatherdata = weather.filter(col("datetime") > "2023-12-31")

In [8]:
futuredata.coalesce(1).write.csv("futuredata", header=True)

In [9]:
futureweatherdata.coalesce(1).write.csv("futureweatherdata", header=True)

In [19]:
pastweatherdata = weather.filter(col("datetime") < "2024-01-01") \
                  .filter(col("datetime") > "2022-12-31")

In [24]:
pastdata = data.filter(col("started_at") < "2024-01-01") \
            .filter(col("started_at") > "2022-12-31") \
            .orderBy(col("started_at").asc()) \
            .dropna(subset=["start_station_id", "end_station_id"]) \
            .withColumn("date", to_date("started_at"))

In [33]:
pastdata = pastdata.groupBy("date").count()

In [34]:
joined = pastdata.join(pastweatherdata, pastdata.date==pastweatherdata.datetime, "inner")

In [36]:
joined.orderBy(col("date").asc()).take(5)

[Row(date=datetime.date(2023, 1, 1), count=6310, datetime=datetime.date(2023, 1, 1), tempmax=16.3, tempmin=6.4, humidity=82.2, precip=0.101, snow=0.0, windspeed=14.2, windgust=22.3, conditions='Rain, Partially cloudy'),
 Row(date=datetime.date(2023, 1, 2), count=6795, datetime=datetime.date(2023, 1, 2), tempmax=17.2, tempmin=4.4, humidity=79.3, precip=0.0, snow=0.0, windspeed=14.7, windgust=16.6, conditions='Partially cloudy'),
 Row(date=datetime.date(2023, 1, 3), count=7077, datetime=datetime.date(2023, 1, 3), tempmax=20.0, tempmin=10.1, humidity=78.5, precip=0.0, snow=0.0, windspeed=28.5, windgust=47.1, conditions='Overcast'),
 Row(date=datetime.date(2023, 1, 4), count=7242, datetime=datetime.date(2023, 1, 4), tempmax=18.3, tempmin=11.9, humidity=85.4, precip=0.0, snow=0.0, windspeed=22.8, windgust=50.0, conditions='Partially cloudy'),
 Row(date=datetime.date(2023, 1, 5), count=7892, datetime=datetime.date(2023, 1, 5), tempmax=17.8, tempmin=9.8, humidity=70.4, precip=0.0, snow=0.0, w

In [37]:
joined.coalesce(1).write.csv("pastdata", header=True)